# ETL Process

In [5]:
from fitizens_libraries.load_data import load_labeled_data
import os

folder_path = "labeled"
os.makedirs(folder_path, exist_ok=True)
file_names = [f"{folder_path}/{name}" for name in os.listdir(folder_path)]

data = load_labeled_data(file_names,
                         ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX",
                          "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"])
df = data[0]["data"]
df

,accX,accY,accZ,gyroX,gyroY,gyroZ,magnX,magnY,magnZ,linAccX,...,magnY_mod,magnZ_mod,linAccX_mod,linAccY_mod,linAccZ_mod,peaks,first_sample,last_sample,first_sample_closest_peak,last_sample_closest_peak
time,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00.000,0.229719,9.351484,2.597228,2.80,-3.85,-2.17,118.426224,91.767258,137.435638,-0.109653,...,8421.229584,18888.554710,0.012024,0.001934,0.009913,NaN,False,False,False,False
1970-01-01 00:00:00.009,0.311078,9.401736,2.568513,2.31,-4.20,-2.17,118.233528,91.525688,136.693176,-0.075312,...,8376.951595,18685.024439,0.005672,0.001504,0.003150,NaN,False,False,False,False
1970-01-01 00:00:00.018,0.366115,9.428058,2.613979,2.17,-4.55,-2.17,117.496239,95.366234,137.098541,-0.007308,...,9094.718554,18796.010016,0.000053,0.002185,0.000273,-0.016523,False,False,False,False
1970-01-01 00:00:00.027,0.327828,9.394557,2.642693,2.73,-5.25,-2.31,117.116760,91.684502,134.205460,-0.005169,...,8406.047842,18011.105385,0.000027,0.000067,0.001790,NaN,False,False,False,False
1970-01-01 00:00:00.036,0.390044,9.423272,2.678587,2.52,-5.67,-2.38,114.384750,90.533600,132.057144,0.061284,...,8196.332702,17439.089325,0.003756,0.000128,0.000008,-0.002824,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:01:17.788,-0.268006,9.664956,1.288307,1.26,0.63,-1.33,121.241829,96.616379,139.818405,0.141657,...,9334.724649,19549.186419,0.020067,0.000266,0.002941,NaN,False,False,False,False
1970-01-01 00:01:17.797,-0.196218,9.736743,1.197377,1.40,0.42,-1.19,121.182777,98.483658,140.009613,0.133558,...,9699.030861,19602.691743,0.017838,0.019219,0.000008,NaN,False,False,False,False
1970-01-01 00:01:17.806,-0.186647,9.765458,1.144733,1.05,-0.07,-1.12,121.378441,97.125923,140.048004,0.106217,...,9433.444949,19613.443467,0.011282,0.034712,0.000599,0.024472,False,False,False,False


In [6]:
df[ df.first_sample_closest_peak == True ]

,accX,accY,accZ,gyroX,gyroY,gyroZ,magnX,magnY,magnZ,linAccX,...,magnY_mod,magnZ_mod,linAccX_mod,linAccY_mod,linAccZ_mod,peaks,first_sample,last_sample,first_sample_closest_peak,last_sample_closest_peak
time,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00.492,0.198611,3.503217,-0.659520,128.169998,-11.480000,-22.960001,117.239410,100.269485,144.137619,-0.081313,...,10053.969717,20775.653216,0.006612,0.265978,39.403793,-6.277244,False,False,True,False
1970-01-01 00:00:03.012,-0.143574,4.532168,-1.509002,114.869995,-13.020000,-19.740000,117.029160,100.863571,146.347122,-0.671627,...,10173.459989,21417.480174,0.451082,0.503256,26.326902,-5.130975,True,False,True,False
1970-01-01 00:00:05.684,-0.241684,3.771223,-1.106994,117.459999,-17.639999,-17.430000,115.418739,104.074860,148.048660,-0.497655,...,10831.576405,21918.405810,0.247660,0.090138,34.982261,-5.914580,False,False,True,False
1970-01-01 00:00:08.378,-0.241684,3.084458,-1.798544,114.939995,-23.030001,-15.050000,118.108246,100.696831,147.712204,-1.027327,...,10139.851723,21818.895204,1.055400,0.619663,41.897643,-6.472839,False,False,True,False
1970-01-01 00:00:11.185,-0.181861,4.993999,-1.714792,111.649994,-6.580000,-21.420000,117.232140,101.666870,149.712448,-0.570794,...,10336.152479,22413.817122,0.325806,0.267923,20.996020,-4.582141,False,False,True,False
1970-01-01 00:00:14.152,-0.028715,4.749922,-1.039992,104.089996,-13.160000,-16.100000,115.861427,101.090431,148.497955,-0.287225,...,10219.275283,22051.642735,0.082498,0.183894,24.739627,-4.973895,False,False,True,False
1970-01-01 00:00:16.784,-0.112467,4.099051,-1.138101,112.279999,-5.600000,-19.670000,114.659546,104.990730,147.374985,-0.318620,...,11023.053446,21719.386127,0.101519,0.164668,31.191293,-5.584917,False,False,True,False
1970-01-01 00:00:19.277,0.583870,3.972228,-1.188352,100.799995,3.010000,-20.090000,116.753517,106.243652,148.984909,-0.030975,...,11287.713663,22196.503127,0.000959,1.157751,33.207800,-5.762621,True,False,True,False
1970-01-01 00:00:22.119,-0.483367,3.866939,-1.726757,103.949997,-6.860000,-16.870001,117.220619,103.823929,152.478043,-0.745809,...,10779.408198,23249.553476,0.556231,0.082479,31.611471,-5.622408,False,False,True,False


In [1]:
import pandas as pd
import os
import zipfile
import json

In [4]:
# Ruta principal que contiene las carpetas comprimidas
C:\Users\57301\Documents\MASTER_IE\TERM_III\FITIZENS\GitHub\mbd23_squats\Descomprimidos
ruta_principal = 'C:/Users/57301/Documents/MASTER_IE/TERM_III/FITIZENS/GitHub/mbd23_squats/LABELED'

# Lista para almacenar los datos de los archivos JSON
datos_json = []

# Recorre cada carpeta comprimida en la carpeta principal
for archivo_zip in os.listdir(ruta_principal):
    if archivo_zip.endswith('.zip'):
        ruta_zip = os.path.join(ruta_principal, archivo_zip)
        
        # Extraer la carpeta comprimida a una ubicación temporal
        carpeta_temporal = 'C:/Users/57301/Documents/MASTER_IE/TERM_III/FITIZENS/GitHub/mbd23_squats/Descomprimidos'
        with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
            zip_ref.extractall(carpeta_temporal)
        
        # Buscar archivos JSON dentro de la carpeta extraída
        for archivo_json in os.listdir(carpeta_temporal):
            if archivo_json.endswith('.json'):
                ruta_json = os.path.join(carpeta_temporal, archivo_json)
                
                # Leer el archivo JSON y agregar los datos a la lista
                with open(ruta_json, 'r') as f:
                    json_data = json.load(f)
                    datos_json.extend(json_data['data'])
        
        # Limpiar la carpeta temporal después de procesar la carpeta comprimida
        for archivo_temporal in os.listdir(carpeta_temporal):
            ruta_temporal = os.path.join(carpeta_temporal, archivo_temporal)
            os.remove(ruta_temporal)

# Convertir la lista de datos JSON a un DataFrame
df = pd.DataFrame(datos_json)

SyntaxError: unexpected character after line continuation character (2134422486.py, line 2)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Check null values
((df.isnull().sum())/len(df))*100

In [ ]:
#Convertir la columna time en datetime object
df['time'] = pd.to_datetime(df['time'])
#check:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

# EDA AND VISUALIZATION

In [ ]:
import seaborn as sns
sns.countplot(x=df['exercise'], label = "Exercise")

In [ ]:
import seaborn as sns
sns.pairplot(df, hue = 'exercise' , vars = ['linAccX','linAccY','linAccZ'])

In [ ]:
# primeros 300 rows: 
prim_300 = df.head(300)
fig = px.line(prim_300, x='time', y='linAccX', title='Time serie of exercise linAccX first 300',color='exercise')
fig.show()

In [ ]:
siguientes_300 = df.iloc[300:600]
fig = px.line(siguientes_300, x='time', y='linAccX', title='Time serie of exercise linAccX second 300',color='exercise')
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# Crear un gráfico con Plotly Express
fig = px.line(df, x='time', y='linAccX', title='Time serie of exercise linAccX', color='exercise')

# Configurar el gráfico para agregar líneas verticales en el inicio del ejercicio 'SQUAT'
for index, row in df.iterrows():
    if row['exercise'] == 'SQUAT':
        fig.add_shape(
            dict(
                type='line',
                x0=row['time'],
                x1=row['time'],
                y0=0,
                y1=300,
                line=dict(color='red', width=2, dash='dash'),
            )
        )

# Actualizar el diseño del gráfico
fig.update_layout(
    shapes=[
        dict(
            type='line',
            x0=row['time'],
            x1=row['time'],
            y0=0,
            y1=300,
            line=dict(color='red', width=2, dash='dash'),
        )
        for index, row in df.iterrows() if row['exercise'] == 'SQUAT'
    ]
)

# Mostrar el gráfico de Plotly Express
fig.show()

# Mostrar el gráfico de matplotlib
fig, ax = plt.subplots(figsize=(10, 6))
for index, row in df.iterrows():
    if row['exercise'] == 'SQUAT':
        ax.axvline(row['time'], color='red', linestyle='--', alpha=0.5)

# Configuración del gráfico de matplotlib
ax.set_title('Serie de Tiempo con Inicio de Ejercicio')
ax.set_xlabel('Tiempo')
ax.set_ylabel('Target')
ax.legend()
plt.show()

In [ ]:
fig = px.line(df, x='milisegundos', y='linAccX', title='Time serie of exercise linAccX',color='exercise')
fig.show()

In [ ]:
import plotly.express as px

#squad
fig = px.line(df, x='time', y='linAccX', title='Time serie of exercise linAccX',color='exercise')
fig.show()

In [ ]:
fig = px.line(df, x='time', y='linAccY', title='Time serie of exercise linAccY',color='exercise')
fig.show()